In [125]:
# !pip install pandas mysql-connector-python
# !pip install sqlalchemy
# !pip install pymysql
import pandas as pd
import numpy as np
import mysql.connector 
from mysql.connector import Error
from urllib.parse import quote_plus
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os
import re

# Normal Transformation and cleaning of the Data

## Establishing connection with MySQL and getting the data

In [2]:
load_dotenv()
user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
port = os.getenv('DB_PORT')
db_staging_1 = os.getenv('DB_STAGING_1')
db_staging_2 = os.getenv('DB_STAGING_2')

In [3]:
read_engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{db_staging_1}')

In [4]:
df = pd.read_sql('amazon_sales', con = read_engine)
df

,uniq_id,product_name,manufacturer,price,purchase_date,ship_date,ship_country,number_available_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews
0,0016eb63fa6c7a5e8930bc7732b13116,Resident Evil 6 Play Arts Kai Helena Harper Ac...,Unknown,£49.88,4/22/2016,4/27/2016,United Kingdom,12 new,1,1,5.0 out of 5 stars,Figures & Playsets > Science Fiction & Fantasy,http://www.amazon.co.uk/Resident-Evil-Kennedy-...,Product Description From Square Enix! From the...,Technical Details Item Weight454 g Product Dim...,Product Description From Square Enix! From the...,http://www.amazon.co.uk/Resident-Evil-Revelati...,,Good figure // 5.0 // 30 Jan. 2014 // By\n ...
1,00182f9b6081e58fea1fe76bc13eb022,Disney Frozen Musical and Light-Up Elsa Fancy ...,George,£24.99,5/19/2018,5/20/2018,Canada,3 new,3,1,4.3 out of 5 stars,Characters & Brands > Disney > Toys,http://www.amazon.co.uk/Disney-Frozen-Elsas-Mu...,"LET IT go, let it go, cant hold it back any mo...",Technical Details Item Weight399 g Product Dim...,"LET IT go, let it go, cant hold it back any mo...",,,Five Stars // 5.0 // 8 Jan. 2015 // By\n \n...
2,0030f5965cbd07a138147425b9a0156c,Disney The Lion King Childrens 24 Piece Simba ...,King International,£6.53,3/4/2016,3/9/2016,Germany,3 new,1,1,5.0 out of 5 stars,Characters & Brands > Disney > Toys,http://www.amazon.co.uk/Ravensburger | http://...,,Technical Details Manufacturer recommended age...,,http://www.amazon.co.uk/Disney-King-4-In-1-Puz...,,Five Stars // 5.0 // 2 Mar. 2015 // By\n \n...
3,0034d77b68130bba71aca6e733c7e1ac,Pokemon Black and White Voice Activated Talkin...,Takara Tomy,£30.23,10/7/2016,10/11/2016,United Kingdom,3 new,1,1,5.0 out of 5 stars,Characters & Brands > Tomy,http://www.amazon.co.uk/Pokemon-Oshawott-Plush...,Pokemon Black and White Voice Activated Talkin...,Technical Details Manufacturer recommended age...,Pokemon Black and White Voice Activated Talkin...,,,great // 5.0 // 10 Jun. 2013 // By\n \n ...
4,0039295c7fcefba10d5fc333eff806d3,LEGO City Airport 60022: Cargo Terminal,LEGO,£199.18,5/3/2016,5/5/2016,United Kingdom,16 new,105,1,4.7 out of 5 stars,,http://www.amazon.co.uk/LEGO-City-Airport-6002...,Product Description Get busy loading the cargo...,Technical Details Item Weight1.8 Kg Product Di...,Product Description Get busy loading the cargo...,http://www.amazon.co.uk/LEGO-60095-Explorers-E...,,Lego City Cargo Terminal // 4.0 // 18 Nov. 201...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,ffd2a93b6cd2b793e288a4b12369d186,K'nex Collect & Build Racecar Rally Series Sto...,K'Nex,£19.99,1/19/2016,1/26/2016,United Kingdom,2 new,1,5,4.0 out of 5 stars,Characters & Brands > Tomy,http://www.amazon.co.uk/Collect-Racecar-Series...,"Collect and Build the blue Stock Car, #1 of 4 ...",Technical Details Item Weight726 g Product Dim...,"Collect and Build the blue Stock Car, #1 of 4 ...",http://www.amazon.co.uk/Collect-Racecar-Series...,,May I have another one // 4.0 // 20 Feb. 2011 ...
9995,ffdb648830521b6d63eceedd976bb770,TY Oriel the Fish Beanie Baby,TY~BEANIES AQUATIC,£6.23,9/25/2016,10/3/2016,United Kingdom,6 new,1,1,5.0 out of 5 stars,,,TY Oriel the Fish Beanie Baby,Technical Details Item Weight68 g Product Dime...,TY Oriel the Fish Beanie Baby,,,Wonderful // 5.0 // 17 Dec. 2012 // By\n \n...
9996,ffe21aa483f78f260ced31907e3a0807,MyTinyWorld Dolls House Miniature 3 Piece Ging...,MyTinyWorld,£4.50,4/2/2016,4/9/2016,Germany,,1,1,5.0 out of 5 stars,Dolls & Accessories > Dolls' House Dolls & Acc...,http://www.amazon.co.uk/MyTinyWorld | http://w...,This range of high quality reusable silicone r...,Technical Details Item Weight5 g Product Dimen...,This range of high quality reusable silicone r...,,,Five Stars // 5.0 // 9 Jun. 2015 // By\n \n...
9997,

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 19 columns):
 #   Column                                       Non-Null Count  Dtype 
---  ------                                       --------------  ----- 
 0   uniq_id                                      9999 non-null   object
 1   product_name                                 9999 non-null   object
 2   manufacturer                                 9999 non-null   object
 3   price                                        9999 non-null   object
 4   purchase_date                                9999 non-null   object
 5   ship_date                                    9999 non-null   object
 6   ship_country                                 9999 non-null   object
 7   number_available_in_stock                    9999 non-null   object
 8   number_of_reviews                            9999 non-null   object
 9   number_of_answered_questions                 9999 non-null   object
 10  average_revi

In [6]:
df.drop(columns=['description'],axis=1, inplace=True)
# dropping this column because it's the same as product_description, it's duplicated

In [7]:
df.head()

,uniq_id,product_name,manufacturer,price,purchase_date,ship_date,ship_country,number_available_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews
0,0016eb63fa6c7a5e8930bc7732b13116,Resident Evil 6 Play Arts Kai Helena Harper Ac...,Unknown,£49.88,4/22/2016,4/27/2016,United Kingdom,12 new,1,1,5.0 out of 5 stars,Figures & Playsets > Science Fiction & Fantasy,http://www.amazon.co.uk/Resident-Evil-Kennedy-...,Technical Details Item Weight454 g Product Dim...,Product Description From Square Enix! From the...,http://www.amazon.co.uk/Resident-Evil-Revelati...,,Good figure // 5.0 // 30 Jan. 2014 // By\n ...
1,00182f9b6081e58fea1fe76bc13eb022,Disney Frozen Musical and Light-Up Elsa Fancy ...,George,£24.99,5/19/2018,5/20/2018,Canada,3 new,3,1,4.3 out of 5 stars,Characters & Brands > Disney > Toys,http://www.amazon.co.uk/Disney-Frozen-Elsas-Mu...,Technical Details Item Weight399 g Product Dim...,"LET IT go, let it go, cant hold it back any mo...",,,Five Stars // 5.0 // 8 Jan. 2015 // By\n \n...
2,0030f5965cbd07a138147425b9a0156c,Disney The Lion King Childrens 24 Piece Simba ...,King International,£6.53,3/4/2016,3/9/2016,Germany,3 new,1,1,5.0 out of 5 stars,Characters & Brands > Disney > Toys,http://www.amazon.co.uk/Ravensburger | http://...,Technical Details Manufacturer recommended age...,,http://www.amazon.co.uk/Disney-King-4-In-1-Puz...,,Five Stars // 5.0 // 2 Mar. 2015 // By\n \n...
3,0034d77b68130bba71aca6e733c7e1ac,Pokemon Black and White Voice Activated Talkin...,Takara Tomy,£30.23,10/7/2016,10/11/2016,United Kingdom,3 new,1,1,5.0 out of 5 stars,Characters & Brands > Tomy,http://www.amazon.co.uk/Pokemon-Oshawott-Plush...,Technical Details Manufacturer recommended age...,Pokemon Black and White Voice Activated Talkin...,,,great // 5.0 // 10 Jun. 2013 // By\n \n ...
4,0039295c7fcefba10d5fc333eff806d3,LEGO City Airport 60022: Cargo Terminal,LEGO,£199.18,5/3/2016,5/5/2016,United Kingdom,16 new,105,1,4.7 out of 5 stars,,http://www.amazon.co.uk/LEGO-City-Airport-6002...,Technical Details Item Weight1.8 Kg Product Di...,Product Description Get busy loading the cargo...,http://www.amazon.co.uk/LEGO-60095-Explorers-E...,,Lego City Cargo Terminal // 4.0 // 18 Nov. 201...


## Transform 'number_available_in_stock' column

In [8]:
# Splitting the number and condition from the 'number_available_in_stock' column
df.insert(6, 'count_in_stock', df['number_available_in_stock'].str.extract(r'(\d+)'))
df.insert(7, 'condition', df['number_available_in_stock'].str.extract(r'(\D+)'))

In [9]:
df.drop(columns='number_available_in_stock', axis=1, inplace=True) # dropping the old column

In [10]:
df.head(2)

,uniq_id,product_name,manufacturer,price,purchase_date,ship_date,count_in_stock,condition,ship_country,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews
0,0016eb63fa6c7a5e8930bc7732b13116,Resident Evil 6 Play Arts Kai Helena Harper Ac...,Unknown,£49.88,4/22/2016,4/27/2016,12,new,United Kingdom,1,1,5.0 out of 5 stars,Figures & Playsets > Science Fiction & Fantasy,http://www.amazon.co.uk/Resident-Evil-Kennedy-...,Technical Details Item Weight454 g Product Dim...,Product Description From Square Enix! From the...,http://www.amazon.co.uk/Resident-Evil-Revelati...,,Good figure // 5.0 // 30 Jan. 2014 // By\n ...
1,00182f9b6081e58fea1fe76bc13eb022,Disney Frozen Musical and Light-Up Elsa Fancy ...,George,£24.99,5/19/2018,5/20/2018,3,new,Canada,3,1,4.3 out of 5 stars,Characters & Brands > Disney > Toys,http://www.amazon.co.uk/Disney-Frozen-Elsas-Mu...,Technical Details Item Weight399 g Product Dim...,"LET IT go, let it go, cant hold it back any mo...",,,Five Stars // 5.0 // 8 Jan. 2015 // By\n \n...


## Cleaning the Price

In [11]:
def clean_price(price):
    # Ensure the price is a string
    if isinstance(price, str):
        # Remove any non-numeric characters, except for the dot
        cleaned_price = price.replace('£', '').replace(',', '')
        try:
            # Convert to float
            return float(cleaned_price)
        except ValueError:
            return None
    return price

In [12]:
df['price'] = df['price'].apply(clean_price) # cleaning the price with the previous function

In [13]:
df.price.sum()

173088.76999999996

In [14]:
df.head(2)

,uniq_id,product_name,manufacturer,price,purchase_date,ship_date,count_in_stock,condition,ship_country,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews
0,0016eb63fa6c7a5e8930bc7732b13116,Resident Evil 6 Play Arts Kai Helena Harper Ac...,Unknown,49.88,4/22/2016,4/27/2016,12,new,United Kingdom,1,1,5.0 out of 5 stars,Figures & Playsets > Science Fiction & Fantasy,http://www.amazon.co.uk/Resident-Evil-Kennedy-...,Technical Details Item Weight454 g Product Dim...,Product Description From Square Enix! From the...,http://www.amazon.co.uk/Resident-Evil-Revelati...,,Good figure // 5.0 // 30 Jan. 2014 // By\n ...
1,00182f9b6081e58fea1fe76bc13eb022,Disney Frozen Musical and Light-Up Elsa Fancy ...,George,24.99,5/19/2018,5/20/2018,3,new,Canada,3,1,4.3 out of 5 stars,Characters & Brands > Disney > Toys,http://www.amazon.co.uk/Disney-Frozen-Elsas-Mu...,Technical Details Item Weight399 g Product Dim...,"LET IT go, let it go, cant hold it back any mo...",,,Five Stars // 5.0 // 8 Jan. 2015 // By\n \n...


## Average review rating

In [15]:
# we will take the rating value only, we don't need that '4.9 out of 5 stars' thing, we simply need the 4.9 only
df['average_review_rating'] = df['average_review_rating'].str.replace('[^\d\.]', '', regex=True).replace('', np.nan).astype(float)

In [16]:
# i don't know why, but after the previous step, the 5.0 values transformed to 5.05
# so that's why im doing this step here, to make them 5.0 again
df['average_review_rating'] = df['average_review_rating'].apply(lambda x: 5 if x > float(5) else x)

In [17]:
df.head(2)

,uniq_id,product_name,manufacturer,price,purchase_date,ship_date,count_in_stock,condition,ship_country,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews
0,0016eb63fa6c7a5e8930bc7732b13116,Resident Evil 6 Play Arts Kai Helena Harper Ac...,Unknown,49.88,4/22/2016,4/27/2016,12,new,United Kingdom,1,1,5.00,Figures & Playsets > Science Fiction & Fantasy,http://www.amazon.co.uk/Resident-Evil-Kennedy-...,Technical Details Item Weight454 g Product Dim...,Product Description From Square Enix! From the...,http://www.amazon.co.uk/Resident-Evil-Revelati...,,Good figure // 5.0 // 30 Jan. 2014 // By\n ...
1,00182f9b6081e58fea1fe76bc13eb022,Disney Frozen Musical and Light-Up Elsa Fancy ...,George,24.99,5/19/2018,5/20/2018,3,new,Canada,3,1,4.35,Characters & Brands > Disney > Toys,http://www.amazon.co.uk/Disney-Frozen-Elsas-Mu...,Technical Details Item Weight399 g Product Dim...,"LET IT go, let it go, cant hold it back any mo...",,,Five Stars // 5.0 // 8 Jan. 2015 // By\n \n...


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 19 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   uniq_id                                      9999 non-null   object 
 1   product_name                                 9999 non-null   object 
 2   manufacturer                                 9999 non-null   object 
 3   price                                        8547 non-null   float64
 4   purchase_date                                9999 non-null   object 
 5   ship_date                                    9999 non-null   object 
 6   count_in_stock                               7499 non-null   object 
 7   condition                                    7499 non-null   object 
 8   ship_country                                 9999 non-null   object 
 9   number_of_reviews                            9999 non-null   object 
 10  

## Transform 'amazon_category_and_sub_category' column

In [19]:
df['category'] = df['amazon_category_and_sub_category'].str.split(' > ').str[0]
df.head(2)

,uniq_id,product_name,manufacturer,price,purchase_date,ship_date,count_in_stock,condition,ship_country,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews,category
0,0016eb63fa6c7a5e8930bc7732b13116,Resident Evil 6 Play Arts Kai Helena Harper Ac...,Unknown,49.88,4/22/2016,4/27/2016,12,new,United Kingdom,1,1,5.00,Figures & Playsets > Science Fiction & Fantasy,http://www.amazon.co.uk/Resident-Evil-Kennedy-...,Technical Details Item Weight454 g Product Dim...,Product Description From Square Enix! From the...,http://www.amazon.co.uk/Resident-Evil-Revelati...,,Good figure // 5.0 // 30 Jan. 2014 // By\n ...,Figures & Playsets
1,00182f9b6081e58fea1fe76bc13eb022,Disney Frozen Musical and Light-Up Elsa Fancy ...,George,24.99,5/19/2018,5/20/2018,3,new,Canada,3,1,4.35,Characters & Brands > Disney > Toys,http://www.amazon.co.uk/Disney-Frozen-Elsas-Mu...,Technical Details Item Weight399 g Product Dim...,"LET IT go, let it go, cant hold it back any mo...",,,Five Stars // 5.0 // 8 Jan. 2015 // By\n \n...,Characters & Brands


In [20]:
categories_df = pd.DataFrame(df['category'].unique())
categories_df.rename(columns={0:'category'}, inplace=True)
categories_df['category'] = categories_df['category'].replace({'':'Unknown'})
categories_df.head()

,category
0,Figures & Playsets
1,Characters & Brands
2,Unknown
3,Games
4,Die-Cast & Toy Vehicles


In [21]:
categories_df.insert(0, 'id', categories_df.index + 1)
categories_df.head()

,id,category
0,1,Figures & Playsets
1,2,Characters & Brands
2,3,Unknown
3,4,Games
4,5,Die-Cast & Toy Vehicles


In [22]:
categories_map = categories_df.set_index('category')['id'].to_dict()
# categories_map

In [23]:
df.insert(3, 'category_id', df['category'].map(categories_map))
df.head(2)

,uniq_id,product_name,manufacturer,category_id,price,purchase_date,ship_date,count_in_stock,condition,ship_country,...,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews,category
0,0016eb63fa6c7a5e8930bc7732b13116,Resident Evil 6 Play Arts Kai Helena Harper Ac...,Unknown,1.0,49.88,4/22/2016,4/27/2016,12,new,United Kingdom,...,1,5.00,Figures & Playsets > Science Fiction & Fantasy,http://www.amazon.co.uk/Resident-Evil-Kennedy-...,Technical Details Item Weight454 g Product Dim...,Product Description From Square Enix! From the...,http://www.amazon.co.uk/Resident-Evil-Revelati...,,Good figure // 5.0 // 30 Jan. 2014 // By\n ...,Figures & Playsets
1,00182f9b6081e58fea1fe76bc13eb022,Disney Frozen Musical and Light-Up Elsa Fancy ...,George,2.0,24.99,5/19/2018,5/20/2018,3,new,Canada,...,1,4.35,Characters & Brands > Disney > Toys,http://www.amazon.co.uk/Disney-Frozen-Elsas-Mu...,Technical Details Item Weight399 g Product Dim...,"LET IT go, let it go, cant hold it back any mo...",,,Five Stars // 5.0 // 8 Jan. 2015 // By\n \n...,Characters & Brands


In [24]:
df['category_id'].isna().sum()

690

In [25]:
df['category_id'] = pd.to_numeric(df['category_id'], downcast='integer', errors='coerce')
df['category_id'] = df['category_id'].fillna(0).astype(int)
df['category_id'] = df['category_id'].replace(0, np.nan).astype('Int64')

In [26]:
df.head(2)

,uniq_id,product_name,manufacturer,category_id,price,purchase_date,ship_date,count_in_stock,condition,ship_country,...,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews,category
0,0016eb63fa6c7a5e8930bc7732b13116,Resident Evil 6 Play Arts Kai Helena Harper Ac...,Unknown,1,49.88,4/22/2016,4/27/2016,12,new,United Kingdom,...,1,5.00,Figures & Playsets > Science Fiction & Fantasy,http://www.amazon.co.uk/Resident-Evil-Kennedy-...,Technical Details Item Weight454 g Product Dim...,Product Description From Square Enix! From the...,http://www.amazon.co.uk/Resident-Evil-Revelati...,,Good figure // 5.0 // 30 Jan. 2014 // By\n ...,Figures & Playsets
1,00182f9b6081e58fea1fe76bc13eb022,Disney Frozen Musical and Light-Up Elsa Fancy ...,George,2,24.99,5/19/2018,5/20/2018,3,new,Canada,...,1,4.35,Characters & Brands > Disney > Toys,http://www.amazon.co.uk/Disney-Frozen-Elsas-Mu...,Technical Details Item Weight399 g Product Dim...,"LET IT go, let it go, cant hold it back any mo...",,,Five Stars // 5.0 // 8 Jan. 2015 // By\n \n...,Characters & Brands


In [27]:
df.drop(columns='category', axis=1, inplace=True)

In [28]:
sub_categories_df = pd.DataFrame(df['amazon_category_and_sub_category'].unique()) # getting the unique categories
sub_categories_df.rename(columns={0:'amazon_category_and_sub_category'}, inplace=True) # so here the column will be named 0
sub_categories_df
# that's why im renaming it, nothing too special here

,amazon_category_and_sub_category
0,Figures & Playsets > Science Fiction & Fantasy
1,Characters & Brands > Disney > Toys
2,Characters & Brands > Tomy
3,
4,Games > Educational Games
...,...
251,Games > Travel & Pocket Games
252,Characters & Brands > LeapFrog > LeapPad Learn...
253,Home Accessories > Seasonal Décor > Christmas ...
254,Dolls & Accessories > Dolls' House Dolls & Acc...


In [29]:
# sub_categories_df['amazon_category_and_sub_category'].str.split(' > ').str[0]

In [30]:
sub_categories_df = sub_categories_df['amazon_category_and_sub_category'].str.split(' > ', expand=True)
sub_categories_df
# here im splitting by '>' to get the category and the sub-categories

,0,1,2,3,4
0,Figures & Playsets,Science Fiction & Fantasy,None,None,None
1,Characters & Brands,Disney,Toys,None,None
2,Characters & Brands,Tomy,None,None,None
3,,None,None,None,None
4,Games,Educational Games,None,None,None
...,...,...,...,...,...
251,Games,Travel & Pocket Games,None,None,None
252,Characters & Brands,LeapFrog,LeapPad Learning System,None,None
253,Home Accessories,Seasonal Décor,Christmas,Novelty Decorations,None
254,Dolls & Accessories,Dolls' House Dolls & Accessories,Dolls' Houses,None,None


In [31]:
sub_categories_df.rename(columns={0:'category', 1:'sub_category_1', 2:'sub_category_2',
                   3:'sub_category_3', 4:'sub_category_4', 5:'sub_category_1'}, inplace=True)
sub_categories_df['category'] = sub_categories_df['category'].replace({'':'Unknown'})
# again, renaming

In [32]:
sub_categories_df.insert(0, 'id', sub_categories_df.index + 1)
# adding a id for each category

In [33]:
sub_categories_df['combined_categories'] = sub_categories_df[['category', 'sub_category_1', 'sub_category_2',
                                                      'sub_category_3', 'sub_category_4']].apply(lambda x: ' > '.join(x.dropna()), axis=1)
# here im combining the values again, because i want to compare it with the origianl DataFrame
# and then put the corresponding 'id' instead of the value in 'amazon_category_and_sub_category'

In [34]:
# sub_category_map = sub_categories_df.set_index('combined_categories')['id'].to_dict()
# here im taking each category and it's id and putting them in a dictionary

In [35]:
# df.insert(4, 'sub_category_id', df['amazon_category_and_sub_category'].map(sub_category_map))
# here im getting the id of each 'amazon_category_and_sub_category' by comaring it's value with the dictionary that was created before this
# and then putthing this id in a new column called 'category_id'
# now i have like a reference to the categories teble

In [36]:
# df['sub_category_id'] = pd.to_numeric(df['sub_category_id'], downcast='integer', errors='coerce')
# df['sub_category_id'] = df['sub_category_id'].fillna(0).astype(int)
# df['sub_category_id'] = df['sub_category_id'].replace(0, np.nan).astype('Int64')
# after inserting the 'id', it was inserted as a float, so here im just casting it to integer again

In [37]:
df.drop(columns='amazon_category_and_sub_category', axis=1, inplace=True) # dropping the old column

In [38]:
df.head(2)

,uniq_id,product_name,manufacturer,category_id,price,purchase_date,ship_date,count_in_stock,condition,ship_country,number_of_reviews,number_of_answered_questions,average_review_rating,customers_who_bought_this_item_also_bought,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews
0,0016eb63fa6c7a5e8930bc7732b13116,Resident Evil 6 Play Arts Kai Helena Harper Ac...,Unknown,1,49.88,4/22/2016,4/27/2016,12,new,United Kingdom,1,1,5.00,http://www.amazon.co.uk/Resident-Evil-Kennedy-...,Technical Details Item Weight454 g Product Dim...,Product Description From Square Enix! From the...,http://www.amazon.co.uk/Resident-Evil-Revelati...,,Good figure // 5.0 // 30 Jan. 2014 // By\n ...
1,00182f9b6081e58fea1fe76bc13eb022,Disney Frozen Musical and Light-Up Elsa Fancy ...,George,2,24.99,5/19/2018,5/20/2018,3,new,Canada,3,1,4.35,http://www.amazon.co.uk/Disney-Frozen-Elsas-Mu...,Technical Details Item Weight399 g Product Dim...,"LET IT go, let it go, cant hold it back any mo...",,,Five Stars // 5.0 // 8 Jan. 2015 // By\n \n...


#### and now i have my Categories table

In [39]:
categories_df.head()

,id,category
0,1,Figures & Playsets
1,2,Characters & Brands
2,3,Unknown
3,4,Games
4,5,Die-Cast & Toy Vehicles


#### and my Sub-Categories table also

In [40]:
sub_categories_df.drop(columns='combined_categories', axis=1, inplace=True)
# dropping the 'combined_categories' column, we don't need it anymore
# we used it just for the sake of inserting the category_id in the original DataFrame

In [41]:
sub_categories_df['category'] = sub_categories_df['category'].map(categories_map)
sub_categories_df.rename(columns={'category':'category_id'}, inplace=True)

In [42]:
sub_categories_df.head(5)

,id,category_id,sub_category_1,sub_category_2,sub_category_3,sub_category_4
0,1,1,Science Fiction & Fantasy,None,None,None
1,2,2,Disney,Toys,None,None
2,3,2,Tomy,None,None,None
3,4,3,None,None,None,None
4,5,4,Educational Games,None,None,None


In [43]:
# sub_category_df = categories_df[['sub_category_1', 'sub_category_2', 'sub_category_3', 'sub_category_4']]
# sub_category_df.insert(0, 'id', categories_df.index + 1)
# sub_category_df.sample(5)

## Manufacturer

In [44]:
manufacturer_df = pd.DataFrame(df['manufacturer'].unique()) # getting the unique manufacturer
manufacturer_df.rename(columns={0:'manufacturer'}, inplace=True) # as before, renaming
# manufacturer_df['manufacturer'] = manufacturer_df['manufacturer'].replace({'':'Unknown'})
manufacturer_df.head(3)

,manufacturer
0,Unknown
1,George
2,King International


In [45]:
manufacturer_df.insert(0, 'id', manufacturer_df.index + 1) # generating an id for each manufacturer
# manufacturer_df['manufacturer'] = manufacturer_df['manufacturer'].replace({'':'Unknown'})
manufacturer_df.head(3)

,id,manufacturer
0,1,Unknown
1,2,George
2,3,King International


In [46]:
manufacturer_map = manufacturer_df.set_index('manufacturer')['id'].to_dict()
# manufacturer_map
# here im taking each manufacturer and it's id and putting them in a dictionary, same as before

In [47]:
df.insert(2, 'manufacturer_id', df['manufacturer'].map(manufacturer_map))
# insert a new 'manufacturer_id' column, containing the corresponding id of each maufacturer

In [48]:
df.drop(columns='manufacturer', axis=1, inplace=True) # droping the old column

In [49]:
df.head(2)

,uniq_id,product_name,manufacturer_id,category_id,price,purchase_date,ship_date,count_in_stock,condition,ship_country,number_of_reviews,number_of_answered_questions,average_review_rating,customers_who_bought_this_item_also_bought,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews
0,0016eb63fa6c7a5e8930bc7732b13116,Resident Evil 6 Play Arts Kai Helena Harper Ac...,1,1,49.88,4/22/2016,4/27/2016,12,new,United Kingdom,1,1,5.00,http://www.amazon.co.uk/Resident-Evil-Kennedy-...,Technical Details Item Weight454 g Product Dim...,Product Description From Square Enix! From the...,http://www.amazon.co.uk/Resident-Evil-Revelati...,,Good figure // 5.0 // 30 Jan. 2014 // By\n ...
1,00182f9b6081e58fea1fe76bc13eb022,Disney Frozen Musical and Light-Up Elsa Fancy ...,2,2,24.99,5/19/2018,5/20/2018,3,new,Canada,3,1,4.35,http://www.amazon.co.uk/Disney-Frozen-Elsas-Mu...,Technical Details Item Weight399 g Product Dim...,"LET IT go, let it go, cant hold it back any mo...",,,Five Stars // 5.0 // 8 Jan. 2015 // By\n \n...


I have a manifacturer table too

In [50]:
manufacturer_df.sample(5)

,id,manufacturer
822,823,SGM
1209,1210,Universal Picture
2156,2157,Mutant Mania
1518,1519,A Nightmare On Elm Street
866,867,Enter Play


## Countries

did the same as for the manufacturer and the categories

In [51]:
countries_df = pd.DataFrame(df['ship_country'].unique())
countries_df.rename(columns={0:'ship_country'}, inplace=True)
countries_df

,ship_country
0,United Kingdom
1,Canada
2,Germany
3,France
4,Australia
5,United States


In [52]:
countries_df.insert(0, 'id', countries_df.index + 1)
countries_df

,id,ship_country
0,1,United Kingdom
1,2,Canada
2,3,Germany
3,4,France
4,5,Australia
5,6,United States


this is my coutries table

In [53]:
countries_map = countries_df.set_index('ship_country')['id'].to_dict()
countries_map

{'United Kingdom': 1,
 'Canada': 2,
 'Germany': 3,
 'France': 4,
 'Australia': 5,
 'United States': 6}

In [54]:
df.insert(7, 'ship_country_id', df['ship_country'].map(countries_map))

In [55]:
df.drop(columns='ship_country', axis=1, inplace=True)

In [56]:
df.head(2)

,uniq_id,product_name,manufacturer_id,category_id,price,purchase_date,ship_date,ship_country_id,count_in_stock,condition,number_of_reviews,number_of_answered_questions,average_review_rating,customers_who_bought_this_item_also_bought,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews
0,0016eb63fa6c7a5e8930bc7732b13116,Resident Evil 6 Play Arts Kai Helena Harper Ac...,1,1,49.88,4/22/2016,4/27/2016,1,12,new,1,1,5.00,http://www.amazon.co.uk/Resident-Evil-Kennedy-...,Technical Details Item Weight454 g Product Dim...,Product Description From Square Enix! From the...,http://www.amazon.co.uk/Resident-Evil-Revelati...,,Good figure // 5.0 // 30 Jan. 2014 // By\n ...
1,00182f9b6081e58fea1fe76bc13eb022,Disney Frozen Musical and Light-Up Elsa Fancy ...,2,2,24.99,5/19/2018,5/20/2018,2,3,new,3,1,4.35,http://www.amazon.co.uk/Disney-Frozen-Elsas-Mu...,Technical Details Item Weight399 g Product Dim...,"LET IT go, let it go, cant hold it back any mo...",,,Five Stars // 5.0 // 8 Jan. 2015 // By\n \n...


## Product Information

In [57]:
import re

# Define a function to extract recommended age using regex
def extract_recommended_age(text):
    if isinstance(text, str):  # Check if text is a string
        match = re.search(r'recommended age:(\d+)', text)
        return match.group(1) if match else None
    return None  # Return None if text is not a string

# Apply the function to the 'product_information' column
# df['recommended_age'] = df['product_information'].apply(extract_recommended_age)
df.insert(7, 'recommended_age', df['product_information'].apply(extract_recommended_age))

In [58]:
# df.drop(columns='product_information', axis=1, inplace=True)

In [59]:
df.head()

,uniq_id,product_name,manufacturer_id,category_id,price,purchase_date,ship_date,recommended_age,ship_country_id,count_in_stock,condition,number_of_reviews,number_of_answered_questions,average_review_rating,customers_who_bought_this_item_also_bought,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews
0,0016eb63fa6c7a5e8930bc7732b13116,Resident Evil 6 Play Arts Kai Helena Harper Ac...,1,1,49.88,4/22/2016,4/27/2016,12,1,12,new,1,1,5.00,http://www.amazon.co.uk/Resident-Evil-Kennedy-...,Technical Details Item Weight454 g Product Dim...,Product Description From Square Enix! From the...,http://www.amazon.co.uk/Resident-Evil-Revelati...,,Good figure // 5.0 // 30 Jan. 2014 // By\n ...
1,00182f9b6081e58fea1fe76bc13eb022,Disney Frozen Musical and Light-Up Elsa Fancy ...,2,2,24.99,5/19/2018,5/20/2018,10,2,3,new,3,1,4.35,http://www.amazon.co.uk/Disney-Frozen-Elsas-Mu...,Technical Details Item Weight399 g Product Dim...,"LET IT go, let it go, cant hold it back any mo...",,,Five Stars // 5.0 // 8 Jan. 2015 // By\n \n...
2,0030f5965cbd07a138147425b9a0156c,Disney The Lion King Childrens 24 Piece Simba ...,3,2,6.53,3/4/2016,3/9/2016,3,3,3,new,1,1,5.00,http://www.amazon.co.uk/Ravensburger | http://...,Technical Details Manufacturer recommended age...,,http://www.amazon.co.uk/Disney-King-4-In-1-Puz...,,Five Stars // 5.0 // 2 Mar. 2015 // By\n \n...
3,0034d77b68130bba71aca6e733c7e1ac,Pokemon Black and White Voice Activated Talkin...,4,2,30.23,10/7/2016,10/11/2016,13,1,3,new,1,1,5.00,http://www.amazon.co.uk/Pokemon-Oshawott-Plush...,Technical Details Manufacturer recommended age...,Pokemon Black and White Voice Activated Talkin...,,,great // 5.0 // 10 Jun. 2013 // By\n \n ...
4,0039295c7fcefba10d5fc333eff806d3,LEGO City Airport 60022: Cargo Terminal,5,<NA>,199.18,5/3/2016,5/5/2016,6,1,16,new,105,1,4.75,http://www.amazon.co.uk/LEGO-City-Airport-6002...,Technical Details Item Weight1.8 Kg Product Di...,Product Description Get busy loading the cargo...,http://www.amazon.co.uk/LEGO-60095-Explorers-E...,,Lego City Cargo Terminal // 4.0 // 18 Nov. 201...


In [60]:
df['recommended_age'].isna().sum()

516

In [61]:
# Replaces numerical ages with categorical labels based on age ranges (Kids, Teenagers, Adults) or returns None for missing values.
def replace_age(age):
    if pd.isna(age):
        return None
    elif int(age) >= 0 and int(age) < 13:
        return 'Kids'
    elif int(age) >= 13 and int(age) < 20:
        return 'Teenagers'
    else:
        return 'Adults'

In [62]:
df.insert(7, 'target_group', df['recommended_age'].apply(replace_age))

In [63]:
df['target_group'].value_counts()

target_group
Kids         7689
Teenagers    1457
Adults        337
Name: count, dtype: int64

In [64]:
df.drop(columns='recommended_age', axis=1, inplace=True)

## Transform the date

In [65]:
df['ship_date'] = pd.to_datetime(df['ship_date'])
df['purchase_date'] = pd.to_datetime(df['purchase_date'])

In [66]:
df.rename(columns={'uniq_id': 'id'}, inplace=True)

In [67]:
df.insert(7, 'shipping_days', (df['ship_date'] - df['purchase_date']).dt.days) # calculate the shipping time in days

In [68]:
# Replaces numerical days with categorical labels based on age ranges (Fast, Standard, Slow)
def categorize_shipping_time(days):
  if days <= 3:
    return 'Fast'
  elif days <= 7:
    return 'Standard'
  else:
    return 'Slow'

df.insert(8, 'shipping_status', df['shipping_days'].apply(categorize_shipping_time))

In [69]:
df.drop(columns='shipping_days', axis=1, inplace=True)

In [70]:
df.head(2)

,id,product_name,manufacturer_id,category_id,price,purchase_date,ship_date,shipping_status,target_group,ship_country_id,...,condition,number_of_reviews,number_of_answered_questions,average_review_rating,customers_who_bought_this_item_also_bought,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews
0,0016eb63fa6c7a5e8930bc7732b13116,Resident Evil 6 Play Arts Kai Helena Harper Ac...,1,1,49.88,2016-04-22,2016-04-27,Standard,Kids,1,...,new,1,1,5.00,http://www.amazon.co.uk/Resident-Evil-Kennedy-...,Technical Details Item Weight454 g Product Dim...,Product Description From Square Enix! From the...,http://www.amazon.co.uk/Resident-Evil-Revelati...,,Good figure // 5.0 // 30 Jan. 2014 // By\n ...
1,00182f9b6081e58fea1fe76bc13eb022,Disney Frozen Musical and Light-Up Elsa Fancy ...,2,2,24.99,2018-05-19,2018-05-20,Fast,Kids,2,...,new,3,1,4.35,http://www.amazon.co.uk/Disney-Frozen-Elsas-Mu...,Technical Details Item Weight399 g Product Dim...,"LET IT go, let it go, cant hold it back any mo...",,,Five Stars // 5.0 // 8 Jan. 2015 // By\n \n...


# Studying With NLP

### Studying the similarity between 'the product_name' and the other items that was bought with this product

In [71]:
import difflib

In [72]:
df.columns

Index(['id', 'product_name', 'manufacturer_id', 'category_id', 'price',
       'purchase_date', 'ship_date', 'shipping_status', 'target_group',
       'ship_country_id', 'count_in_stock', 'condition', 'number_of_reviews',
       'number_of_answered_questions', 'average_review_rating',
       'customers_who_bought_this_item_also_bought', 'product_information',
       'product_description', 'items_customers_buy_after_viewing_this_item',
       'customer_questions_and_answers', 'customer_reviews'],
      dtype='object')

In [73]:
# combining the values of 'customers_who_bought_this_item_also_bought' and 'items_customers_buy_after_viewing_this_item'
def combine_without_similars(row):
    items = []
    if pd.notna(row['customers_who_bought_this_item_also_bought']):
        items.extend(row['customers_who_bought_this_item_also_bought'].split('|'))
    if pd.notna(row['items_customers_buy_after_viewing_this_item']):
        items.extend(row['items_customers_buy_after_viewing_this_item'].split('|'))

    # Remove similar sentences
    unique_items = []
    for item in items:
        if not any(difflib.SequenceMatcher(None, item, unique_item).ratio() > 0.8 for unique_item in unique_items):
            unique_items.append(item)
    
    return '|'.join(unique_items)

# Apply function to each row
df.insert(15, 'also_bought', df.apply(combine_without_similars, axis=1))

In [74]:
df['also_bought'].iloc[2]

'http://www.amazon.co.uk/Ravensburger | http://www.amazon.co.uk/Disney-Princess-Childrens-Horses-Jigsaw/dp/B00BQZ3K8M | http://www.amazon.co.uk/Disney-King-4-In-1-Puzzles-Pieces/dp/B00CAUYKI6 | http://www.amazon.co.uk/Two-By-DVD-Chris-Evans/dp/B00WZ70PKQ | http://www.amazon.co.uk/There-Swallowed-Classic-Books-Holes/dp/0859537277 | http://www.amazon.co.uk/Melissa-Doug-Dinosaurs-Puzzles-Box/dp/B0033UWNDQ| http://www.amazon.co.uk/Disney-King-Childrens-Everyone-Together/dp/B00BQZC376 | http://www.amazon.co.uk/Orchard-Toys-216-Whos-Jungle/dp/B0007TXTL6 | http://www.amazon.co.uk/Lion-King-Read-Along-Storybook-Audio/dp/142313768X'

In [75]:
also_bought_df = df[['id','also_bought']]

In [76]:
# Function to extract words from URLs
def extract_words_from_urls(urls):
    if pd.isna(urls):
        return ''
    urls_list = urls.split('|')
    words = [re.search(r'http://www\.amazon\.co\.uk/(.*)', url).group(1).replace('/dp/', '-') for url in urls_list if re.search(r'http://www\.amazon\.co\.uk/(.*)', url)]
    words = [word.strip() for word in words]
    return ' | '.join(words)

df['also_bought'] = df['also_bought'].apply(extract_words_from_urls)

In [77]:
df['also_bought'].iloc[2]

'Ravensburger | Disney-Princess-Childrens-Horses-Jigsaw-B00BQZ3K8M | Disney-King-4-In-1-Puzzles-Pieces-B00CAUYKI6 | Two-By-DVD-Chris-Evans-B00WZ70PKQ | There-Swallowed-Classic-Books-Holes-0859537277 | Melissa-Doug-Dinosaurs-Puzzles-Box-B0033UWNDQ | Disney-King-Childrens-Everyone-Together-B00BQZC376 | Orchard-Toys-216-Whos-Jungle-B0007TXTL6 | Lion-King-Read-Along-Storybook-Audio-142313768X'

In [78]:
# Function to remove the suffix from each item in the 'also_bought' column
def remove_suffix(also_bought):
    if pd.isna(also_bought):
        return ''
    words = also_bought.split(' | ')
    cleaned_words = [re.sub(r'-B\w+$', '', word) for word in words]
    cleaned_words = [re.sub(r'-[0-9]+[a-zA-Z]*', '', word) for word in cleaned_words]
    return ' | '.join(cleaned_words)

df['also_bought'] = df['also_bought'].apply(remove_suffix)

In [79]:
df['also_bought'].iloc[2]

'Ravensburger | Disney-Princess-Childrens-Horses-Jigsaw | Disney-King-In-Puzzles-Pieces | Two-By-DVD-Chris-Evans | There-Swallowed-Classic-Books-Holes | Melissa-Doug-Dinosaurs-Puzzles-Box | Disney-King-Childrens-Everyone-Together | Orchard-Toys-Whos-Jungle | Lion-King-Read-Along-Storybook-Audio'

In [80]:
# Function to calculate similarity using the difflib library
def calculate_similarity(product_name, also_bought):
    if pd.isna(also_bought):
        return 0
    urls_list = also_bought.split(' | ')
    max_similarity = 0
    for url in urls_list:
        similarity = difflib.SequenceMatcher(None, product_name, url).ratio()
        if similarity > max_similarity:
            max_similarity = similarity
    return max_similarity * 100  # Convert to percentage

df.insert(17, 'also_bought_similarity', df.apply(lambda row: calculate_similarity(row['product_name'], row['also_bought']), axis=1))
df['also_bought_similarity']=df['also_bought_similarity'].round(2)

In [81]:
df[['product_name','also_bought','also_bought_similarity']].sample(10)

,product_name,also_bought,also_bought_similarity
5716,Red Tartan Scottish Stag Suit (Men: XX-Large),Psychedelic | Test-Screen-Officially | Rainbow...,27.85
9641,Kit ricambi - Toys Action figures MATTEL,Tyco-MICRO-SLOTCAR-FIREBIRD | Tyco-TYCO-cliff-...,17.86
415,LEGO® Star WarsTM Minifig Watto (from set 7962),LEGO-Star-Wars-TM | LEGO-Star-Wars-Princess-Wh...,48.28
674,Lamborghini Diablo Coupe VT (Red),,0.00
4689,Hornby Railroad 00 Gauge LNER Class A4 Gadwell...,Hornby-R8243-Surface-Mounted-Point | Hornby-Ra...,43.33
3201,"Fireman Sam Filled Party Bags (no. 2), THREE s...",Fireman-Sam-Birthday-Party-Banner | Fireman-Sa...,48.84
4993,13.75 Inch Walnut and Maple Chess Board,Inlaid-wooden-chess-board-No | 12-Walnut-Maple...,66.67
7899,Hot Wheels Jukebox 1:64 Scale Die-cast Vehicle...,Wheels-Twenty-Random-Cars-Models,24.78
7589,Gamewright Feed the Kitty,Gamewright,57.14
5175,Carrera Go Formula Duel,Carrera-Go-Extension-Set | Stadlbauer | James-...,48.00


In [82]:
df.drop(columns=['also_bought'], axis=1, inplace=True)

In [83]:
df.head(2)

,id,product_name,manufacturer_id,category_id,price,purchase_date,ship_date,shipping_status,target_group,ship_country_id,...,number_of_reviews,number_of_answered_questions,average_review_rating,customers_who_bought_this_item_also_bought,also_bought_similarity,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews
0,0016eb63fa6c7a5e8930bc7732b13116,Resident Evil 6 Play Arts Kai Helena Harper Ac...,1,1,49.88,2016-04-22,2016-04-27,Standard,Kids,1,...,1,1,5.00,http://www.amazon.co.uk/Resident-Evil-Kennedy-...,60.87,Technical Details Item Weight454 g Product Dim...,Product Description From Square Enix! From the...,http://www.amazon.co.uk/Resident-Evil-Revelati...,,Good figure // 5.0 // 30 Jan. 2014 // By\n ...
1,00182f9b6081e58fea1fe76bc13eb022,Disney Frozen Musical and Light-Up Elsa Fancy ...,2,2,24.99,2018-05-19,2018-05-20,Fast,Kids,2,...,3,1,4.35,http://www.amazon.co.uk/Disney-Frozen-Elsas-Mu...,45.38,Technical Details Item Weight399 g Product Dim...,"LET IT go, let it go, cant hold it back any mo...",,,Five Stars // 5.0 // 8 Jan. 2015 // By\n \n...


In [84]:
df.loc[df['customer_questions_and_answers'] != ''].shape

(913, 22)

### Studying the customer reviews

In [85]:
import joblib
model = joblib.load('model.pkl') # loading the trained model
vectorizer = joblib.load('tfidf_vectorizer.pkl')

C:\Users\MYCOM\miniconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.4.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\MYCOM\miniconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.4.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\MYCOM\miniconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.4.2 when using version 1.5.1. This might lead t

In [86]:
# Pre-Process the customers reviews column
def extract_tex(text):
    if isinstance(text, str):
        text = re.sub(r'\W+', ' ', text).split()
        text = ' '.join(text)
        text = re.sub(r'\d+', '', text)
    return text  # Return the text
# df.insert(19, 'customer_reviews_2', df['customer_reviews'].apply(extract_tex))
df['customer_reviews_2'] = df['customer_reviews'].apply(extract_tex)
df['customer_reviews_2']

0       Good figure    Jan  By Xray and vav on  Jan  T...
1       Five Stars    Jan  By sandra siert on  Jan  Gr...
2       Five Stars    Mar  By Charles parker on  Mar  ...
3       great    Jun  By louise on  Jun  this is a gre...
4       Lego City Cargo Terminal    Nov  By LR on  Nov...
                              ...                        
9994    May I have another one    Feb  By Mrs Porkybik...
9995    Wonderful    Dec  By APC on  Dec  This Fish lo...
9996        Five Stars    Jun  By June on  Jun  excellent
9997    a gift    Mar  By helen wiltshire on  Mar  As ...
9998    Fun for the WHOLE family    Jan  By colwobs on...
Name: customer_reviews_2, Length: 9999, dtype: object

In [87]:
from scipy.sparse import hstack

df['customer_reviews_2'] = df['customer_reviews_2'].fillna('')
reviews = vectorizer.transform(df['customer_reviews_2'])

labels = model.predict(reviews)

In [88]:
df['label'] = labels

In [89]:
df[['customer_reviews_2','label']].sample(10)

,customer_reviews_2,label
6134,received this lovely train today and I am real...,1
6150,Five Stars July By jb on July looked gre...,1
106,fab July By jack on July My son loved th...,1
5060,great toy July By Ms Sally A Leathers on ...,1
7835,Burger Boss gives you plenty to get your teeth...,1
4023,Lovely Jun By Chloe Huang on Jun Easy to...,1
9582,Clean easy to assemble kit for a nice price ...,1
2971,Brilliant July of people found the follo...,1
354,Five Stars Mar By T C on Mar Look pretty...,1
2877,Brilliant toy Really fuels a childs imaginatio...,1


In [90]:
df.columns

Index(['id', 'product_name', 'manufacturer_id', 'category_id', 'price',
       'purchase_date', 'ship_date', 'shipping_status', 'target_group',
       'ship_country_id', 'count_in_stock', 'condition', 'number_of_reviews',
       'number_of_answered_questions', 'average_review_rating',
       'customers_who_bought_this_item_also_bought', 'also_bought_similarity',
       'product_information', 'product_description',
       'items_customers_buy_after_viewing_this_item',
       'customer_questions_and_answers', 'customer_reviews',
       'customer_reviews_2', 'label'],
      dtype='object')

In [91]:
df.drop(columns='customer_reviews_2', axis=1, inplace=True)

In [92]:
df.head(2)

,id,product_name,manufacturer_id,category_id,price,purchase_date,ship_date,shipping_status,target_group,ship_country_id,...,number_of_answered_questions,average_review_rating,customers_who_bought_this_item_also_bought,also_bought_similarity,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews,label
0,0016eb63fa6c7a5e8930bc7732b13116,Resident Evil 6 Play Arts Kai Helena Harper Ac...,1,1,49.88,2016-04-22,2016-04-27,Standard,Kids,1,...,1,5.00,http://www.amazon.co.uk/Resident-Evil-Kennedy-...,60.87,Technical Details Item Weight454 g Product Dim...,Product Description From Square Enix! From the...,http://www.amazon.co.uk/Resident-Evil-Revelati...,,Good figure // 5.0 // 30 Jan. 2014 // By\n ...,1
1,00182f9b6081e58fea1fe76bc13eb022,Disney Frozen Musical and Light-Up Elsa Fancy ...,2,2,24.99,2018-05-19,2018-05-20,Fast,Kids,2,...,1,4.35,http://www.amazon.co.uk/Disney-Frozen-Elsas-Mu...,45.38,Technical Details Item Weight399 g Product Dim...,"LET IT go, let it go, cant hold it back any mo...",,,Five Stars // 5.0 // 8 Jan. 2015 // By\n \n...,1


# Splitting Transformations

### Also bought table

In [93]:
also_bought_df

,id,also_bought
0,0016eb63fa6c7a5e8930bc7732b13116,http://www.amazon.co.uk/Resident-Evil-Kennedy-...
1,00182f9b6081e58fea1fe76bc13eb022,http://www.amazon.co.uk/Disney-Frozen-Elsas-Mu...
2,0030f5965cbd07a138147425b9a0156c,http://www.amazon.co.uk/Ravensburger | http://...
3,0034d77b68130bba71aca6e733c7e1ac,http://www.amazon.co.uk/Pokemon-Oshawott-Plush...
4,0039295c7fcefba10d5fc333eff806d3,http://www.amazon.co.uk/LEGO-City-Airport-6002...
...,...,...
9994,ffd2a93b6cd2b793e288a4b12369d186,http://www.amazon.co.uk/Collect-Racecar-Series...
9995,ffdb648830521b6d63eceedd976bb770,
9996,ffe21aa483f78f260ced31907e3a0807,http://www.amazon.co.uk/MyTinyWorld |
9997,ffecb46876e79436e3c687f0fce0d0b3,http://www.amazon.co.uk/Vilac-2932-Giant-Fish-...


In [94]:
# Split URLs and expand into separate rows
def split_urls(row):
    urls = row['also_bought'].split('|')
    return pd.Series({'id': row['id'], 'also_bought': urls})

also_bought_df = also_bought_df.apply(split_urls, axis=1).explode('also_bought').reset_index(drop=True)

also_bought_df

,id,also_bought
0,0016eb63fa6c7a5e8930bc7732b13116,http://www.amazon.co.uk/Resident-Evil-Kennedy-...
1,0016eb63fa6c7a5e8930bc7732b13116,http://www.amazon.co.uk/Batman-Arkham-Origins...
2,0016eb63fa6c7a5e8930bc7732b13116,http://www.amazon.co.uk/Star-Images-Clarke-Ac...
3,0016eb63fa6c7a5e8930bc7732b13116,http://www.amazon.co.uk/Square-Enix-Tomb-Raid...
4,0016eb63fa6c7a5e8930bc7732b13116,http://www.amazon.co.uk/Square-Enix-Street-Fi...
...,...,...
51296,ffee9727cfb5f53e12dd9d9fca0cf8aa,http://www.amazon.co.uk/AK-SPORT-Super-Soaker...
51297,ffee9727cfb5f53e12dd9d9fca0cf8aa,http://www.amazon.co.uk/Nerf-Super-Soaker-Mic...
51298,ffee9727cfb5f53e12dd9d9fca0cf8aa,http://www.amazon.co.uk/Nerf-A5625-Supersoaker
51299,ffee9727cfb5f53e12dd9d9fca0cf8aa,http://www.amazon.co.uk/NERF-Super-Soaker-Bar...


In [95]:
also_bought_df['id'].nunique()

9999

In [96]:
df.drop(columns=['customers_who_bought_this_item_also_bought','items_customers_buy_after_viewing_this_item'], axis=1, inplace=True)

In [97]:
# df = df[['id', 'product_name', 'manufacturer_id', 'category_id', 'price',
#        'purchase_date', 'ship_date', 'shipping_status', 'target_group',
#        'ship_country_id', 'count_in_stock', 'condition', 'number_of_reviews',
#        'number_of_answered_questions', 'average_review_rating',
#        'product_information', 'product_description', 'similarity',
#        'customer_questions_and_answers', 'customer_reviews', 'label']]

In [98]:
df.head(3)

,id,product_name,manufacturer_id,category_id,price,purchase_date,ship_date,shipping_status,target_group,ship_country_id,...,condition,number_of_reviews,number_of_answered_questions,average_review_rating,also_bought_similarity,product_information,product_description,customer_questions_and_answers,customer_reviews,label
0,0016eb63fa6c7a5e8930bc7732b13116,Resident Evil 6 Play Arts Kai Helena Harper Ac...,1,1,49.88,2016-04-22,2016-04-27,Standard,Kids,1,...,new,1,1,5.00,60.87,Technical Details Item Weight454 g Product Dim...,Product Description From Square Enix! From the...,,Good figure // 5.0 // 30 Jan. 2014 // By\n ...,1
1,00182f9b6081e58fea1fe76bc13eb022,Disney Frozen Musical and Light-Up Elsa Fancy ...,2,2,24.99,2018-05-19,2018-05-20,Fast,Kids,2,...,new,3,1,4.35,45.38,Technical Details Item Weight399 g Product Dim...,"LET IT go, let it go, cant hold it back any mo...",,Five Stars // 5.0 // 8 Jan. 2015 // By\n \n...,1
2,0030f5965cbd07a138147425b9a0156c,Disney The Lion King Childrens 24 Piece Simba ...,3,2,6.53,2016-03-04,2016-03-09,Standard,Kids,3,...,new,1,1,5.00,46.30,Technical Details Manufacturer recommended age...,,,Five Stars // 5.0 // 2 Mar. 2015 // By\n \n...,1


### Q&A

In [99]:
df.loc[df['customer_questions_and_answers'] != '', 'customer_questions_and_answers']

21      Will the deck grip fit on a mini micro scooter...
26      Do the doors open? // No they dont | What's th...
32      What material is this made of? // It's made of...
39      Can you tell me the diameter of each spiral an...
66      Can you say why it is half price please? // De...
                              ...                        
9961    what is difference between dx6 and dx6i // Che...
9977    what size/scale is this?   1:16? // Molly its ...
9981    Do the Pat and Jess figures come out of the va...
9986    What's the size of the castle when mounted ? /...
9992    please can you tell me what filters i need for...
Name: customer_questions_and_answers, Length: 913, dtype: object

In [100]:
df['customer_questions_and_answers'].iloc[26]

"Do the doors open? // No they dont | What's the size please? // Not sure of proper measure but it is much bigger than a matchbox toy. It is a lovely item , well made and my grandson loves it. Sorry i cant measure for u as he took it home | hi can you supply corgi olympic taxi destination london 2012 full set of 40 // Yes we can."

In [101]:
qa_df = df[['id','customer_questions_and_answers']].copy()
qa_df['customer_questions_and_answers'] = qa_df['customer_questions_and_answers'].replace({'':None})
qa_df['customer_questions_and_answers'].isna().sum(), qa_df['customer_questions_and_answers'].notna().sum()

(9086, 913)

In [102]:
qa_df = pd.DataFrame(qa_df)
qa_df

,id,customer_questions_and_answers
0,0016eb63fa6c7a5e8930bc7732b13116,None
1,00182f9b6081e58fea1fe76bc13eb022,None
2,0030f5965cbd07a138147425b9a0156c,None
3,0034d77b68130bba71aca6e733c7e1ac,None
4,0039295c7fcefba10d5fc333eff806d3,None
...,...,...
9994,ffd2a93b6cd2b793e288a4b12369d186,None
9995,ffdb648830521b6d63eceedd976bb770,None
9996,ffe21aa483f78f260ced31907e3a0807,None
9997,ffecb46876e79436e3c687f0fce0d0b3,None


In [103]:
# Split Q&A and expand into separate rows
def split_questions(row):
    if pd.isna(row['customer_questions_and_answers']):
        return pd.Series({'id': row['id'], 'customer_questions_and_answers': None})
    qa = row['customer_questions_and_answers'].split('|')
    return pd.Series({'id': row['id'], 'customer_questions_and_answers': qa})

qa_df = qa_df.apply(split_questions, axis=1).explode('customer_questions_and_answers').reset_index(drop=True)

def remove_outer_spaces(text):
    if pd.isna(text):
        return None
    return text.strip()
qa_df['customer_questions_and_answers'] = qa_df['customer_questions_and_answers'].apply(remove_outer_spaces)
qa_df = pd.DataFrame(qa_df)
qa_df

,id,customer_questions_and_answers
0,0016eb63fa6c7a5e8930bc7732b13116,None
1,00182f9b6081e58fea1fe76bc13eb022,None
2,0030f5965cbd07a138147425b9a0156c,None
3,0034d77b68130bba71aca6e733c7e1ac,None
4,0039295c7fcefba10d5fc333eff806d3,None
...,...,...
10758,ffd2a93b6cd2b793e288a4b12369d186,None
10759,ffdb648830521b6d63eceedd976bb770,None
10760,ffe21aa483f78f260ced31907e3a0807,None
10761,ffecb46876e79436e3c687f0fce0d0b3,None


In [104]:
qa_df['id'].nunique()

9999

In [105]:
qa_df.shape

(10763, 2)

In [106]:
qa_df.dropna(inplace=True)
qa_df.shape

(1677, 2)

In [107]:
qa_df.reset_index(inplace=True)
qa_df.rename(columns={'customer_questions_and_answers': 'q_and_a'}, inplace=True)
qa_df.drop(columns='index', axis=1, inplace=True)

In [108]:
qa_df.head(5)

,id,q_and_a
0,00dcc458c7bba7164260c63f4e5143cb,Will the deck grip fit on a mini micro scooter...
1,00f53c984c3f4489569cc86ca4a0acc5,Do the doors open? // No they dont
2,00f53c984c3f4489569cc86ca4a0acc5,What's the size please? // Not sure of proper ...
3,00f53c984c3f4489569cc86ca4a0acc5,hi can you supply corgi olympic taxi destinati...
4,0116b2f3e75dd9e1d3206e5bee76bbae,What material is this made of? // It's made of...


In [109]:
print(qa_df.iloc[1, 1])
print(qa_df.iloc[2, 1])
print(qa_df.iloc[3, 1])

Do the doors open? // No they dont
What's the size please? // Not sure of proper measure but it is much bigger than a matchbox toy. It is a lovely item , well made and my grandson loves it. Sorry i cant measure for u as he took it home
hi can you supply corgi olympic taxi destination london 2012 full set of 40 // Yes we can.


In [110]:
df.drop(columns='customer_questions_and_answers', axis=1, inplace=True)

In [111]:
df.head(2)

,id,product_name,manufacturer_id,category_id,price,purchase_date,ship_date,shipping_status,target_group,ship_country_id,count_in_stock,condition,number_of_reviews,number_of_answered_questions,average_review_rating,also_bought_similarity,product_information,product_description,customer_reviews,label
0,0016eb63fa6c7a5e8930bc7732b13116,Resident Evil 6 Play Arts Kai Helena Harper Ac...,1,1,49.88,2016-04-22,2016-04-27,Standard,Kids,1,12,new,1,1,5.00,60.87,Technical Details Item Weight454 g Product Dim...,Product Description From Square Enix! From the...,Good figure // 5.0 // 30 Jan. 2014 // By\n ...,1
1,00182f9b6081e58fea1fe76bc13eb022,Disney Frozen Musical and Light-Up Elsa Fancy ...,2,2,24.99,2018-05-19,2018-05-20,Fast,Kids,2,3,new,3,1,4.35,45.38,Technical Details Item Weight399 g Product Dim...,"LET IT go, let it go, cant hold it back any mo...",Five Stars // 5.0 // 8 Jan. 2015 // By\n \n...,1


### Product info

In [112]:
# reviews_df.iloc[0,0]

In [113]:
product_info_df = df[['id','product_information', 'product_description']].copy()
product_info_df

,id,product_information,product_description
0,0016eb63fa6c7a5e8930bc7732b13116,Technical Details Item Weight454 g Product Dim...,Product Description From Square Enix! From the...
1,00182f9b6081e58fea1fe76bc13eb022,Technical Details Item Weight399 g Product Dim...,"LET IT go, let it go, cant hold it back any mo..."
2,0030f5965cbd07a138147425b9a0156c,Technical Details Manufacturer recommended age...,
3,0034d77b68130bba71aca6e733c7e1ac,Technical Details Manufacturer recommended age...,Pokemon Black and White Voice Activated Talkin...
4,0039295c7fcefba10d5fc333eff806d3,Technical Details Item Weight1.8 Kg Product Di...,Product Description Get busy loading the cargo...
...,...,...,...
9994,ffd2a93b6cd2b793e288a4b12369d186,Technical Details Item Weight726 g Product Dim...,"Collect and Build the blue Stock Car, #1 of 4 ..."
9995,ffdb648830521b6d63eceedd976bb770,Technical Details Item Weight68 g Product Dime...,TY Oriel the Fish Beanie Baby
9996,ffe21aa483f78f260ced31907e3a0807,Technical Details Item Weight5 g Product Dimen...,This range of high quality reusable silicone r...
9997,ffecb46876e79436e3c687f0fce0d0b3,Technical Details Item Weight132 g Product Dim...,Product Description Lovingly crafted from Vila...


In [114]:
product_info_df['product_information'] = product_info_df['product_information'].replace({'':None}) 
product_info_df['product_description'] = product_info_df['product_description'].replace({'':None}) 

In [115]:
product_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   9999 non-null   object
 1   product_information  9941 non-null   object
 2   product_description  9349 non-null   object
dtypes: object(3)
memory usage: 234.5+ KB


In [116]:
product_info_df

,id,product_information,product_description
0,0016eb63fa6c7a5e8930bc7732b13116,Technical Details Item Weight454 g Product Dim...,Product Description From Square Enix! From the...
1,00182f9b6081e58fea1fe76bc13eb022,Technical Details Item Weight399 g Product Dim...,"LET IT go, let it go, cant hold it back any mo..."
2,0030f5965cbd07a138147425b9a0156c,Technical Details Manufacturer recommended age...,None
3,0034d77b68130bba71aca6e733c7e1ac,Technical Details Manufacturer recommended age...,Pokemon Black and White Voice Activated Talkin...
4,0039295c7fcefba10d5fc333eff806d3,Technical Details Item Weight1.8 Kg Product Di...,Product Description Get busy loading the cargo...
...,...,...,...
9994,ffd2a93b6cd2b793e288a4b12369d186,Technical Details Item Weight726 g Product Dim...,"Collect and Build the blue Stock Car, #1 of 4 ..."
9995,ffdb648830521b6d63eceedd976bb770,Technical Details Item Weight68 g Product Dime...,TY Oriel the Fish Beanie Baby
9996,ffe21aa483f78f260ced31907e3a0807,Technical Details Item Weight5 g Product Dimen...,This range of high quality reusable silicone r...
9997,ffecb46876e79436e3c687f0fce0d0b3,Technical Details Item Weight132 g Product Dim...,Product Description Lovingly crafted from Vila...


In [117]:
df.drop(columns=['product_information', 'product_description'], axis=1, inplace=True)
df.head(2)

,id,product_name,manufacturer_id,category_id,price,purchase_date,ship_date,shipping_status,target_group,ship_country_id,count_in_stock,condition,number_of_reviews,number_of_answered_questions,average_review_rating,also_bought_similarity,customer_reviews,label
0,0016eb63fa6c7a5e8930bc7732b13116,Resident Evil 6 Play Arts Kai Helena Harper Ac...,1,1,49.88,2016-04-22,2016-04-27,Standard,Kids,1,12,new,1,1,5.00,60.87,Good figure // 5.0 // 30 Jan. 2014 // By\n ...,1
1,00182f9b6081e58fea1fe76bc13eb022,Disney Frozen Musical and Light-Up Elsa Fancy ...,2,2,24.99,2018-05-19,2018-05-20,Fast,Kids,2,3,new,3,1,4.35,45.38,Five Stars // 5.0 // 8 Jan. 2015 // By\n \n...,1


# Loading the Data into MySQL

In [118]:
df = df[['id', 'product_name', 'manufacturer_id', 'category_id', 'price',
       'purchase_date', 'ship_date', 'shipping_status', 'target_group',
       'ship_country_id', 'count_in_stock', 'condition', 'number_of_reviews',
       'number_of_answered_questions', 'average_review_rating',
         'also_bought_similarity', 'customer_reviews', 'label']]

In [119]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   id                            9999 non-null   object        
 1   product_name                  9999 non-null   object        
 2   manufacturer_id               9999 non-null   int64         
 3   category_id                   9309 non-null   Int64         
 4   price                         8547 non-null   float64       
 5   purchase_date                 9999 non-null   datetime64[ns]
 6   ship_date                     9999 non-null   datetime64[ns]
 7   shipping_status               9999 non-null   object        
 8   target_group                  9483 non-null   object        
 9   ship_country_id               9999 non-null   int64         
 10  count_in_stock                7499 non-null   object        
 11  condition                     

In [120]:
df.columns

Index(['id', 'product_name', 'manufacturer_id', 'category_id', 'price',
       'purchase_date', 'ship_date', 'shipping_status', 'target_group',
       'ship_country_id', 'count_in_stock', 'condition', 'number_of_reviews',
       'number_of_answered_questions', 'average_review_rating',
       'also_bought_similarity', 'customer_reviews', 'label'],
      dtype='object')

In [121]:
load_engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{db_staging_2}')

In [127]:
with load_engine.begin() as connection:
    connection.execute(text("DELETE FROM product_sales"))
    connection.execute(text("DELETE FROM categories"))
    connection.execute(text("DELETE FROM sub_categories"))
    connection.execute(text("DELETE FROM manufacturer"))
    connection.execute(text("DELETE FROM countries"))
    connection.execute(text("DELETE FROM also_bought"))
    connection.execute(text("DELETE FROM questions_answers"))
    connection.execute(text("DELETE FROM product_info"))

In [129]:
categories_df.to_sql("categories", con = load_engine, if_exists= 'append', index= False)

44

In [130]:
sub_categories_df.to_sql("sub_categories", con = load_engine, if_exists= 'append', index= False)

256

In [131]:
manufacturer_df.to_sql("manufacturer", con = load_engine, if_exists= 'append', index= False)

2653

In [132]:
countries_df.rename(columns={'ship_country':'country'}, inplace=True)
countries_df.to_sql("countries", con = load_engine, if_exists= 'append', index= False)

6

In [133]:
df.to_sql("product_sales", con = load_engine, if_exists= 'append', index= False)

9999

In [134]:
also_bought_df.to_sql("also_bought", con = load_engine, if_exists= 'append', index= False)

51301

In [135]:
qa_df.to_sql("questions_answers", con = load_engine, if_exists= 'append', index= False)

1677

In [136]:
product_info_df.to_sql("product_info", con = load_engine, if_exists= 'append', index= False)

9999